<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import psycopg2

In [4]:
from connection_data import *

In [5]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [27]:
# текст запроса
query_3_1 = f'''
select count(id)
from vacancies
'''

In [28]:
# результат запроса
vacancies_count = pd.read_sql_query(query_3_1, connection)['count'][0]
print(f"Общее кол-во: {vacancies_count} вакансий")

Общее кол-во: 49197 вакансий


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [29]:
# текст запроса
query_3_2 = f'''
select count(id)
from employers
'''

In [30]:
# результат запроса
employers_count = pd.read_sql_query(query_3_2, connection)['count'][0]
print(f"Общее кол-во: {employers_count} работодатель")

Общее кол-во: 23501 работодатель


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [31]:
# текст запроса
query_3_3 = f'''
select count(id)
from areas
'''

In [32]:
# результат запроса
areas_count = pd.read_sql_query(query_3_3, connection)['count'][0]
print(f"Общее кол-во: {areas_count} региона")

Общее кол-во: 1362 региона


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [33]:
# текст запроса
query_3_4 = f'''
select count(id)
from industries
'''

In [34]:
# результат запроса
industries_count = pd.read_sql_query(query_3_4, connection)['count'][0]
print(f"Общее кол-во: {industries_count} сферы деятельности")

Общее кол-во: 294 сферы деятельности


***

Выводы:
1. Общее количество вакансий - 49 тыс.
2. Общее количество работадателей - 23,5 тыс.
3. Общее количество регионов - 1362 региона.
4. Общее количество сфер деятельности - 294.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [35]:
# текст запроса
query_4_1 = f'''
select
    a.name,
    count(v.name) cnt
from vacancies v
    join areas a on a.id = v.area_id
group by a.name
order by cnt desc
'''

In [36]:
# результат запроса
cnt_df = pd.read_sql_query(query_4_1, connection)
display(cnt_df)

,name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [37]:
# текст запроса
query_4_2 = f'''
select
    count(id)
from vacancies
where salary_from is not NULL OR salary_to is not NULL
'''

In [38]:
# результат запроса
salary_count = pd.read_sql_query(query_4_2, connection)['count'][0]
print(f"У {salary_count} вакансий заполнена зарплата")

У 24073 вакансий заполнена зарплата


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [39]:
# текст запроса
query_4_3 = f'''
select
    avg(salary_from) as "from",
    avg(salary_to) as "to"
from vacancies
'''

In [40]:
# результат запроса
avg_salarys = pd.read_sql_query(query_4_3, connection)
print("Нижняя граница: {0}, верхняя граница: {1} заработной платы".format(avg_salarys["from"].round()[0], avg_salarys["to"].round()[0]))

Нижняя граница: 71065.0, верхняя граница: 110537.0 заработной платы


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [41]:
# текст запроса
query_4_4 = f'''
select
    v.schedule,
    v.employment,
    count(v.id)
from vacancies v
group by v.schedule, v.employment
order by count(v.id) desc
'''

In [42]:
# результат запроса
vacancies_count_BySchEmp = pd.read_sql_query(query_4_4, connection)
display(vacancies_count_BySchEmp)

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [43]:
# текст запроса
query_4_5 = f'''
select 
    experience "Требуемый опыт работы"
from vacancies
group by experience
order by count(id)
'''

In [44]:
# результат запроса
experience_must = pd.read_sql_query(query_4_5, connection)
display(experience_must)

,Требуемый опыт работы
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

Выводы:
1. Топ 5 регионов по кол-ву вакансий:
    1. Москва: 5333 вакансии
    2. Санкт-Петербург: 2851
    3. Минск: 2112
    4. Новосибирск: 2006
    5. Алматы: 1892

Такое распределение связано с густым населением регионов, чем больше потенциальных работников имеется в регионе, тем большее количество вакансий будет предложено.

2. Чуть меньше чем у половины вакансий (24 тыс.) заполнена заработная плата.

Вывод: большая половина вакансий ориентирована на сдельную заработную плату.

3. Оплата труда в среднем варьируется от 71 тыс. до 110 тыс. рублей

4. Больше половины вакансий (35 тыс.) предлагают постоянную работу (полный день и полную занятость)

Компаниям не хватает штатных сотрудников.

5. Большинство вакансий ориентированно на молодых специалистов с опытом работы от 1 года до 3 лет

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [45]:
# текст запроса
query_5_1 = f'''
with x as
(
select
    e.name employ_name,
    count(v.id) vacancies_count
from vacancies as v
    join employers as e on e.id = v.employer_id
group by e.name
order by count(v.id) desc
limit 5
)

(select 
    employ_name
from x
limit 1)

union all

(select 
    employ_name
from x
order by vacancies_count
limit 1)
'''

In [46]:
# результат запроса
employ_top = pd.read_sql_query(query_5_1, connection)
display(employ_top)

,employ_name
0,Яндекс
1,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [47]:
# текст запроса
query_5_2 = f'''
select
    a.name area_name,
    count(e.name) employ_cnt,
    count(v.name) vacan_cnt
from areas as a
    left outer join employers as e on e.area = a.id
    left outer join vacancies as v on v.area_id = a.id and v.employer_id = e.id
group by a.name
order by count(v.name), count(e.name) desc
'''

In [48]:
# результат запроса
area_v_e_count = pd.read_sql_query(query_5_2, connection)
display(area_v_e_count)

,area_name,employ_cnt,vacan_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Великобритания,23,0
4,Краснодарский край,19,0
...,...,...,...
1357,Екатеринбург,1166,960
1358,Алматы,1276,1069
1359,Санкт-Петербург,3069,1811
1360,Минск,2228,1858


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [49]:
# текст запроса
query_5_3 = f'''
select
    e.name employer,
    count(distinct a.name) area_cnt
from vacancies as v
    join areas as a on a.id = v.area_id
    join employers as e on e.id = v.employer_id
group by e.name
order by count(a.name) desc
'''

In [50]:
# результат запроса
employ_area_cnt = pd.read_sql_query(query_5_3, connection)
display(employ_area_cnt)

,employer,area_cnt
0,Яндекс,181
1,Ростелеком,152
2,Тинькофф,43
3,СБЕР,24
4,Газпром нефть,24
...,...,...
14761,ПСБ Инновации и Инвестиции,1
14762,ПСЗ Янтарь,1
14763,ПСК,1
14764,Псковский завод механических приводов,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [51]:
# текст запроса
query_5_4 = f'''
select
    count(e.id)
from employers as e
    left outer join employers_industries as ei on ei.employer_id = e.id
where employer_id is NULL
'''

In [52]:
# результат запроса
employer_industryless = pd.read_sql_query(query_5_4, connection)["count"][0]
print(f"У {employer_industryless} работадателей не указана сфера деятельности")

У 8419 работадателей не указана сфера деятельности


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [53]:
# текст запроса
query_5_5 = f'''
select
    e.name
from employers as e
    full outer join employers_industries as ei on ei.employer_id = e.id
group by e.name
having count(ei.industry_id) = 4
order by e.name
offset 2
limit 1
'''

In [54]:
# результат запроса
third_employer = pd.read_sql_query(query_5_5, connection).iloc[0][0]
print(f"Работадатель \"{third_employer}\" указал ровно 4 сферы деятельности и расположен на 3 месте в алфавитном порядке компаний, которые указали столько же сфер деятельности")

Работадатель "2ГИС" указал ровно 4 сферы деятельности и расположен на 3 месте в алфавитном порядке компаний, которые указали столько же сфер деятельности


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [55]:
# текст запроса
query_5_6 = f'''
select
    count(e.name)
from employers as e
    join employers_industries as ei on ei.employer_id = e.id
    join industries as i on i.id = ei.industry_id
where i.name = 'Разработка программного обеспечения'
'''

In [56]:
# результат запроса
employer_it_cnt = pd.read_sql_query(query_5_6, connection)["count"][0]
print(f"{employer_it_cnt} работадателей в сфере разработки программного обеспечения")

3553 работадателей в сфере разработки программного обеспечения


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [57]:
# код для получения списка городов-милионников
def get_city_list(mode="million"):
    '''
    Функция для получения городов миллионников с сайта википедии

    Inputs:
        mode (str, optional): Выборка городов (million, submillion, all). Default to "million"
    
    Outputs:
        list
    '''
    import requests
    from bs4 import BeautifulSoup

    url = r"https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"

    result = requests.get(url)
    if result.status_code == 200:
        page = result.text
    else:
        print("Error: Status code:", result.status_code)
        return []

    soup = BeautifulSoup(page, "html.parser")

    if mode in ("million", "submillion", "all"):
        start, stop = {
            "million": (0, 1),
            "submillion": (1, 2),
            "all": (0, 2)}[mode]
    else:
        start, stop = 0, 1
    
    citys = list()
    for table in soup.find_all("table")[start:stop]:
        for row in table.find_all("tr")[1:]:
            citys.append(row.find_all("td")[1].text.replace("\n", ""))

    return citys

In [58]:
# текст запроса
million_citys = tuple(get_city_list())
query_5_7 = f'''
(select
    a.name city_name,
    count(v.id) cnt
from vacancies as v
    join employers as e on e.id = v.employer_id
    join areas as a on a.id = v.area_id
where e.name = 'Яндекс' and a.name in {million_citys}
group by a.name)

union all

(select
    'Total',
    count(v.id)
from vacancies as v
    join employers as e on e.id = v.employer_id
    join areas as a on a.id = v.area_id
where e.name = 'Яндекс' and a.name in {million_citys})
order by cnt
'''

In [59]:
# результат запроса
yandex_million_city = pd.read_sql_query(query_5_7, connection)
display(yandex_million_city)

,city_name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

Выводы:
1. Среди всех регионов, на первом месте по кол-ву вакансий находится компания ***"Яндекс"***, на пятом ***"Газпром нефть"***.
2. В **России** 410 работодателей и ни одной вакансии.

Вероятно в России прогрессирует дифицид рабочих мест.

3. Топ 3 работадателя с самым широким охватом:
	1. Яндекс: 181 регион с вакансиями;
	2. Ростелеком: 152 региона;
	3. Тинькофф: 43 региона.
4. У 8,5 тыс. работадателей **не указана** сфера деятельности.
5. 3,5 тыс. работадателей работают в сфере `разработки программного обеспечения`.
6. Компания ***"Яндекс"*** всего предлагает 485 вакансий в городах миллионниках.
Наибольшее кол-во вакансий предложено в **Москве** (54 вакансии), меньше всего в **Омске** (21 вакансия) среди городов миллионников.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [60]:
# текст запроса
query_6_1 = f'''
select count(name) vacan_cnt
from vacancies
where lower(name) LIKE '%data%' or lower(name) LIKE '%данн%'
'''

In [61]:
# результат запроса
data_vacan_cnt = pd.read_sql_query(query_6_1, connection).iloc[0][0]
print(f"Всего {data_vacan_cnt} вакансий связанных с данными")

Всего 1771 вакансий связанных с данными


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [62]:
# текст запроса
query_6_2 = f'''
select 
    count(v.name)
from vacancies v
where
    (lower(v.name) LIKE '%data scientist%' OR
    lower(v.name) LIKE '%data science%' OR
    lower(v.name) LIKE '%исследователь данных%' OR
    v.name LIKE '%ML%' and v.name NOT LIKE '%HTML%' and v.name NOT LIKE '%QML%' OR
    lower(v.name) LIKE '%machine learning%' OR
    lower(v.name) LIKE '%машинн%обучен%') 
    AND
    (lower(v.name) LIKE '%junior%' OR
    v.experience LIKE 'Нет опыта' OR
    v.employment LIKE 'Стажировка')
'''

In [63]:
# результат запроса
suitable_vacan_cnt = pd.read_sql_query(query_6_2, connection).iloc[0][0]
print(f"Кол-во подходящих вакансий: {suitable_vacan_cnt}")

Кол-во подходящих вакансий: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [64]:
# текст запроса
query_6_3 = f'''
select 
    count(v.name)
from vacancies v
where
    (lower(v.name) LIKE '%data scientist%' OR
    lower(v.name) LIKE '%data science%' OR
    lower(v.name) LIKE '%исследователь данных%' OR
    lower(v.name) LIKE '%ml%' and v.name NOT LIKE '%HTML%' and v.name NOT LIKE '%QML%' OR
    lower(v.name) LIKE '%machine learning%' OR
    lower(v.name) LIKE '%машинн%обучен%') 
    AND
    (lower(v.key_skills) LIKE '%sql%' or
    lower(v.key_skills) LIKE '%postgres%')
'''

In [65]:
# результат запроса
suitable_vacan_sql_postgres_cnt = pd.read_sql_query(query_6_3, connection).iloc[0][0]
print(f"Кол-во подходящих вакансий: {suitable_vacan_sql_postgres_cnt}")

Кол-во подходящих вакансий: 229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [66]:
# текст запроса
query_6_4 = f'''
select 
    count(v.name)
from vacancies v
where
    (lower(v.name) LIKE '%data scientist%' OR
    lower(v.name) LIKE '%data science%' OR
    lower(v.name) LIKE '%исследователь данных%' OR
    lower(v.name) LIKE '%ml%' and v.name NOT LIKE '%HTML%' and v.name NOT LIKE '%QML%' OR
    lower(v.name) LIKE '%machine learning%' OR
    lower(v.name) LIKE '%машинн%обучен%') 
    AND
    lower(v.key_skills) LIKE '%python%'
'''

In [67]:
# результат запроса
python_vac_cnt = pd.read_sql_query(query_6_4, connection).iloc[0][0]
print(f"Кол-во вакансий для Python разработчика в сфере DS: {python_vac_cnt}")

Кол-во вакансий для Python разработчика в сфере DS: 357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [68]:
# текст запроса
query_6_5 = f'''
select
round(avg(length(key_skills) - length(replace(key_skills,CHR(9),''))+1),2)

from vacancies v

WHERE
(
       (v.name ILIKE '%data scientist%')
    OR (lower(v.name) LIKE '%data science%')
    OR (v.name ILIKE '%исследователь данных%')
    OR (v.name  LIKE '%ML%' AND  (v.name NOT LIKE '%HTML%'))
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
    AND (key_skills IS NOT NULL)
)
'''

In [69]:
# результат запроса
avg_skills_cnt = pd.read_sql_query(query_6_5, connection).iloc[0].round(2)[0]
print(f"В среднем на должность DS нужно обладать {avg_skills_cnt} инструментами")

В среднем на должность DS нужно обладать 6.41 инструментами


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [70]:
# текст запроса
query_6_6 = f'''
with filtered_table as (
    select * 
from vacancies v
where
    (
        (v.name ILIKE '%data scientist%')
        OR (v.name ILIKE '%data science%')
        OR (v.name ILIKE '%исследователь данных%')
        OR (v.name  LIKE '%ML%' AND  (v.name NOT LIKE '%HTML%'))
        OR (v.name ILIKE '%machine learning%')
        OR (v.name ILIKE '%машинн%обучен%')
    ) 
)

select
    experience,
    round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2)) as avg_salary
from filtered_table
where
    salary_from is not NULL or salary_to is not NULL
group by experience

'''

In [71]:
# результат запроса
table = pd.read_sql_query(query_6_6, connection)
table

,experience,avg_salary
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

Выводы:
1. Всего 1771 вакансия, которая связана с работой с данными.
2. Для начинающего (junior) дата саинтиста существует 51 вакансия (около 3% от общего числа вакансий в этом направлении).
3. Количество вакансий в которых требуется знание SQL или PostgreSQL насчитывает 229 (12% от общего числа вакансий).
4. Знание Python указали в 357 вакансиях для работы с данными или 20% от общего числа вакансий.

Python - это быстро прогрессирующий язык программирования, который был создан для анализа данных, поэтому все больше компаний требуют знание данного языка.

5. В среднем на должность связанную с обработкой данных требуется владение 6 - 7 различными инструментами или навыками по работе с данными.
6. Средняя заработная плата указываемая на должность дата саинтиста сгруппированная по опыту работы:
	* От 3 до 6 лет: 243 115 руб.
	* От 1 года до 3 лет: 139 675 руб.
	* Без опыта: 74 643 руб.

# Общий вывод по проекту

В среднем, каждый работодатель предлагает по 2 вакансии. 
Среднее кол-во работодателей в каждом регионе - 17, а на каждую сферу деятельности предлагают в среднем по 167 вакансий. 
Имеется в среднем по 80 работодателей на каждую сферу деятельности.
Проще всего найти подходящую работу в Москве, так как там больше всего вакансий.
Лидирующую позицию на рынке имеет компания "Яндекс". Она имеет самый широкий охват регионов.
Среди всех вакансий, лишь 1771 вакансия имеет отношение к данным, где оплата труда стартует в среднем от 74 тыс. рублей.

# Дополнительные исследования

Проведем дополнительное исследование и дополним наш вывод:

Найдем среднюю оплату труда в Москве, количество вакансий компании "Яндекс" связанные с данными и среднюю оплату труда компании "Яндекс" в Москве в сфере DS.

1. Найти среднюю оплату труда в Москве.

In [72]:
my_query1 = f'''
select
    round(avg((COALESCE(v.salary_to, v.salary_from) + COALESCE(v.salary_from, v.salary_to)) / 2), 2)
from vacancies v
    join areas a on a.id = v.area_id
where
    a.name = 'Москва' and
    (v.salary_to is not NULL or v.salary_from is not NULL)
'''

In [73]:
avg_salary_moscov = pd.read_sql_query(my_query1, connection).iloc[0][0]
print(f"Средняя оплата труда в Москве - {avg_salary_moscov} руб.")

Средняя оплата труда в Москве - 134140.17 руб.


2. Найти количество вакансий компании "Яндекс" связанные с данными.

In [74]:
my_query2 = f'''
with filtered_table as (
    select * 
from vacancies v
where
    (
        (v.name ILIKE '%data scientist%')
        OR (v.name ILIKE '%data science%')
        OR (v.name ILIKE '%исследователь данных%')
        OR (v.name  LIKE '%ML%' AND  (v.name NOT LIKE '%HTML%'))
        OR (v.name ILIKE '%machine learning%')
        OR (v.name ILIKE '%машинн%обучен%')
    ) 
)

select
    count(ft.id)
from filtered_table ft
    join employers e on e.id = ft.employer_id
where
    e.name = 'Яндекс'
'''

In [75]:
cnt_vacan_ds_ya = pd.read_sql_query(my_query2, connection).iloc[0][0]
print(f"Количество вакансий: {cnt_vacan_ds_ya}")

Количество вакансий: 9


3. Найти среднюю оплату труда компании "Яндекс" в Москве в сфере DS

In [76]:
my_query3 = f'''
with filtered_table as (
    select * 
from vacancies v
where
    (
        (v.name ILIKE '%data scientist%')
        OR (v.name ILIKE '%data science%')
        OR (v.name ILIKE '%исследователь данных%')
        OR (v.name  LIKE '%ML%' AND  (v.name NOT LIKE '%HTML%'))
        OR (v.name ILIKE '%machine learning%')
        OR (v.name ILIKE '%машинн%обучен%')
    ) 
)

select
    round(avg((COALESCE(ft.salary_to, ft.salary_from) + COALESCE(ft.salary_from, ft.salary_to)) / 2), 2)
from filtered_table ft
    join employers e on e.id = ft.employer_id
    join areas a on a.id = ft.area_id
where
    e.name = 'Яндекс' and
    a.name = 'Москва' and
    (ft.salary_to is not NULL or ft.salary_from is not NULL)
'''

In [77]:
avg_salary_ds_ya_moscov = pd.read_sql_query(my_query3, connection).iloc[0][0]
print(f"Средняя оплата труда: {avg_salary_ds_ya_moscov}")

Средняя оплата труда: None


___

Выводы:
1. Средняя оплата труда в Москве 134 тыс. рублей.
2. Компания "Яндекс" предлагает 9 вакансий в сфере DS.
3. Компания "Яндекс" не предлагает вакансий в сфере DS в Москве. Либо в данном регионе зарплата не указана.

Проведем ещё одно исследование и узнаем в каких регионах компания "Яндекс" предлагает вакансии в сфере DS, а так же посмотрим на оплату труда.

In [78]:
my_query4 = f'''
with filtered_table as (
    select * 
from vacancies v
where
    (
        (v.name ILIKE '%data scientist%')
        OR (v.name ILIKE '%data science%')
        OR (v.name ILIKE '%исследователь данных%')
        OR (v.name  LIKE '%ML%' AND  (v.name NOT LIKE '%HTML%'))
        OR (v.name ILIKE '%machine learning%')
        OR (v.name ILIKE '%машинн%обучен%')
    ) 
)

select
    a.name,
    ft.salary_from,
    ft.salary_to
from filtered_table ft
    join employers e on e.id = ft.employer_id
    join areas a on a.id = ft.area_id
where
    e.name = 'Яндекс'
'''

In [79]:
ya_vacan_ds = pd.read_sql_query(my_query4, connection)
display(ya_vacan_ds)

,name,salary_from,salary_to
0,Екатеринбург,None,None
1,Москва,None,None
2,Москва,None,None
3,Москва,None,None
4,Санкт-Петербург,None,None
5,Казань,None,None
6,Новосибирск,None,None
7,Москва,None,None
8,Санкт-Петербург,None,None


Таким образом мы выяснили, что помимо других городов, компания "Яндекс" предлагает вакансии в сфере DS в Москве (4 вакансии), но не указывает заработную плату.